In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [ ]:
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
# More complex chain

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat?")

In [ ]:
# Template

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
chain.invoke({"question": "what do bears like to eat?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})

In [ ]:
inputs.invoke({"question": "what do bears like to eat?"})

In [ ]:
chain2 = inputs | prompt | model | output_parser

In [ ]:
chain2.invoke({"question": "what do bears like to eat?"})

In [ ]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for a weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code" : {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                }
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sports events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name" : {
                    "type": "string",
                    "description": "The sports team name search for"
                }
            },
            "required": ["team_name"]
        }
    }
    
]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "what is the weather in NY?"})

In [ ]:
runnable.invoke({"input": "what is latest for trabzon spor?"})

In [ ]:
# Fallbacks 

In [ ]:
from langchain.llms import OpenAI
import json

In [ ]:
simple_model = OpenAI(
    temperature=0,
    max_tokens = 1000,
    model="text-davinci-001"
)
simple_chain = simple_model |json.loads

In [ ]:
challenge = "write three poems in a json blob where each poem is a json blob of a title, author, and the first line"

In [ ]:
simple_model.invoke(challenge)

In [ ]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

In [ ]:
# Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell ma a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": " frogs"}])

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response 